In [4]:
import os
os.environ['HF_HOME']='/mnt/huggingface/'
os.environ['CUDA_VISIBLE_DEVICES']='0'

import chromadb
import chromadb.utils.embedding_functions as embedding_functions


client = chromadb.HttpClient(host='130.216.216.35', port=8001)
client.heartbeat()

1745019057621156699

In [52]:
import json

def get_entity_list(file_name):
    with open(f"../data/entities/{file_name}", 'r') as f:

        return json.load(f)
    

In [53]:
stores = {}

ent_files = ['subjects.json', 'actions.json', 'resources.json', 'conditions.json']

for file in ent_files:
    ent_type = file.split('.')[0]
    ents = get_entity_list(file)
    collection = client.get_or_create_collection(name=f"{ent_type}", embedding_function=embedding_functions.HuggingFaceEmbeddingFunction(model_name="mixedbread-ai/mxbai-embed-large-v1", api_key=""), metadata={"hnsw:space": "cosine"})
    collection.add(documents=ents, ids=[f"id {i+1}" for i in range(len(ents))])
    stores[ent_type] = collection

In [5]:
client.list_collections()

[Collection(name=action),
 Collection(name=nlacps),
 Collection(name=subject),
 Collection(name=resource),
 Collection(name=condition)]

In [58]:
stores['resources'].query(query_texts='medical records',n_results=1)

{'ids': [['id 1']],
 'distances': [[0.041035295]],
 'embeddings': None,
 'metadatas': [[None]],
 'documents': [['medical record']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [41]:
import numpy as np

def get_entities(query, collection, k = 3, return_scores = True):
    res = collection.query(query_texts=query,n_results=k)
    dists = res['distances'][0]
    scores = 1 - np.array(dists)
    if return_scores:
        return res['documents'][0], scores.tolist()
    return res['documents'][0]
        

In [43]:
get_entities('hcp', stores['subjects'], k=1)

(['hcp'], [0.99999982118607])

In [12]:
nlacps = client.get_collection(name = 'nlacps', embedding_function=embedding_functions.HuggingFaceEmbeddingFunction(model_name="mixedbread-ai/mxbai-embed-large-v1", api_key=""))
nlacps

Collection(name=nlacps)

In [14]:
nlacps.query(query_texts=['hcp'], n_results=5)

{'ids': [['e4748ad0-5f5e-478a-828d-83b8df0011d4',
   '2b061f07-4faf-4163-8f8b-4fe288600c00',
   '6cbb32a0-3a4f-4516-b141-d964efbf9aa5',
   '047dadf5-14b7-4729-ad93-6a54fd922ff0',
   '67e72d33-2e49-4556-b65c-f52fa21cc0ea']],
 'distances': [[0.315696, 0.32771683, 0.42898595, 0.46531963, 0.5100448]],
 'embeddings': None,
 'metadatas': [[None, None, None, None, None]],
 'documents': [['Patients can designate or revoke HCP.',
   'Administrators create and assign HCPs.',
   'LHCPs can assign UAPs.',
   'DLHCPs and LHCPs can view approved medical records.',
   'Patients and personal representatives can access full medical records.']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [32]:
client.delete_collection('subjects')
client.delete_collection('actions')
client.delete_collection('resources')
client.delete_collection('conditions')

In [55]:
# pip install bm25s
import bm25s
from dataclasses import dataclass

@dataclass
class Element():
    content: str = "",
    id: int = 0

    @staticmethod
    def from_dict(data: dict):
        return Element(
            content = data['content'],
            id = data['id']
        )

# Create your corpus here
corpus = [
    Element.from_dict({'content': "a cat is a feline and likes to eat fish", "id": 1}),
    Element.from_dict({"content": "a dog is the human's best friend and loves to play", "id": 2}),
    Element.from_dict({"content": "a bird is a beautiful animal that can fly", "id": 3}),
    Element.from_dict({"content": "a fish is a creature that lives in water and swims", "id": 4}),
]

# Create the BM25 model and index the corpus
retriever = bm25s.BM25(corpus=corpus)
retriever.index(bm25s.tokenize([d.content for d in corpus]))

# Query the corpus and get top-k results
query = "cats"
results, scores = retriever.retrieve(bm25s.tokenize(query), k=4)

# # Let's see what we got!
# doc, score = results[0, 0], scores[0, 0]
# print(f"(score: {score:.2f}): {doc}")

results


Split strings:   0%|          | 0/4 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/4 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/4 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

array([[Element(content='a fish is a creature that lives in water and swims', id=4),
        Element(content='a bird is a beautiful animal that can fly', id=3),
        Element(content="a dog is the human's best friend and loves to play", id=2),
        Element(content='a cat is a feline and likes to eat fish', id=1)]],
      dtype=object)

In [56]:
scores

array([[0., 0., 0., 0.]], dtype=float32)

In [52]:
mask = scores >= 0

In [53]:
mask

array([[ True,  True,  True,  True]])

In [54]:
results[mask].tolist()

[Element(content='a fish is a creature that lives in water and swims', id=4),
 Element(content='a cat is a feline and likes to eat fish', id=1),
 Element(content='a bird is a beautiful animal that can fly', id=3),
 Element(content="a dog is the human's best friend and loves to play", id=2)]

In [3]:
from elasticsearch import Elasticsearch

# Create the client instance
client = Elasticsearch("http://localhost:9200")

# Successful response!
client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))))

In [ ]:
client.in